In [1]:
from __future__ import print_function
from sklearn.feature_extraction import FeatureHasher
from pprint import pprint
from time import time
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
def split_X_and_Y(data):
    Y_data = data[['click']]
    del data['click']
    X_data = data

    return X_data, Y_data

In [3]:
def extract_time_stamp_feature(data_frame):
    data_frame['day'] = data_frame['hour'].apply(lambda x: (x - x % 10000) / 1000000)  # day
    data_frame['dow'] = data_frame['hour'].apply(lambda x: ((x - x % 10000) / 1000000) % 7)  # day of week
    data_frame['hour'] = data_frame['hour'].apply(lambda x: x % 10000 / 100)  # hour
    return data_frame

In [4]:
def hashing_data(data_frame, column_name, hasher,number_of_features):
    list_column_name = []
    for i in range(number_of_features):
        list_column_name.append(column_name + str(i))
    new_set_of_columns = hasher.transform(data_frame[column_name].values)
    test = pd.DataFrame(new_set_of_columns.toarray(), columns=list_column_name)
    result = pd.concat([data_frame, test], axis=1)
    del result[column_name]
    return result

In [7]:
data = pd.read_table("C:\\Users\\tuana\\Desktop\\kaggle\\sample_data.csv", sep=',', index_col=False)
original_data=data.copy()
list_hash_columns = ['site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category',
                     'device_model',
                     'device_id', 'device_ip']
list_number_hashing_features=[5]
print(data.head(10))

                     id  click      hour  banner_pos   site_id site_domain  \
0  14192751356771942009      0  14102100           0  85f751fd    c4e18dd6   
1   1661917939641506785      0  14102100           0  85f751fd    c4e18dd6   
2   2657369591670314894      0  14102100           0  85f751fd    c4e18dd6   
3  11533941941013118885      0  14102100           0  1fbe01fe    f3845767   
4   4690542634279392329      0  14102100           0  85f751fd    c4e18dd6   
5  16450495357336661478      0  14102100           0  6256f5b4    28f93029   
6  12005074170819954140      0  14102100           1  e59ef3fc    0a4015b2   
7  12909982014001073860      1  14102100           0  6c5b482c    7687a86e   
8   7237707799313167653      1  14102100           0  543a539e    c7ca3108   
9  15244551246716705914      0  14102100           0  83a0ad1a    5c9ae867   

  site_category    app_id app_domain app_category     ...      \
0      50e219e0  6415a993   2347f47a     0f2161f8     ...       
1      50e2

In [8]:
pipeline = Pipeline([
    ('clf', SGDClassifier(tol=0.000001)),
])

parameters = {
    'clf__alpha': (0.01,0.001,0.0001, 0.00001),
    'clf__learning_rate':(['optimal']),
    # 'clf__eta0':(0.0001, 0.00001,0.000001),
    # 'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__n_iter': (1000, 5000, 8000, 10000),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=2, verbose=1, cv=4)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)

Performing grid search...
pipeline: ['clf']
parameters:
{'clf__alpha': (0.01, 0.001, 0.0001, 1e-05), 'clf__learning_rate': ['optimal']}


In [9]:
for number_hashing_features in list_number_hashing_features:
    print("Number of hashing features : %d" % number_hashing_features)
    data=original_data
    hasher = FeatureHasher(n_features=number_hashing_features, input_type='string')

    for column_name in list_hash_columns:
        data = hashing_data(data, column_name, hasher,number_hashing_features)

    data = extract_time_stamp_feature(data)
    X_data, Y_data = split_X_and_Y(data)

    grid_search.fit(X_data, Y_data['click'])

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    print(grid_search.grid_scores_)
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Number of hashing features : 5
Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=2)]: Done  16 out of  16 | elapsed:  2.0min finished


Best score: 0.830
Best parameters set:
[mean: 0.66511, std: 0.28598, params: {'clf__learning_rate': 'optimal', 'clf__alpha': 0.01}, mean: 0.83023, std: 0.00000, params: {'clf__learning_rate': 'optimal', 'clf__alpha': 0.001}, mean: 0.83023, std: 0.00000, params: {'clf__learning_rate': 'optimal', 'clf__alpha': 0.0001}, mean: 0.66511, std: 0.28598, params: {'clf__learning_rate': 'optimal', 'clf__alpha': 1e-05}]
	clf__alpha: 0.001
	clf__learning_rate: 'optimal'


C:\Users\tuana\Anaconda2\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
